# TensorFlow Quantumの実行サンプル

## TensorFlow Quantum とは

- TensorFlowにCirqを統合し、さらに量子回路のパラメーターをニューラルネットで調整する機能をKeras風味で実装した量子機械学習用ライブラリ

## 注意
- google colabにて動作を確認。その時の各ライブラリのバージョンは
  - tensorflow: 2.1.0  ※ 2.1.0 でないと tensorflow_quantum が エラーで import できません
  - tensorflow_quantum: 0.3.0
  - Cirq:0.8.0 
- 参考資料にあるサイトのコードをコピペしただけで、コードの中身も、合っているかどうかもまだよく知りません（笑）これは「とりあえず実行できているサンプル」というだけです。

## 参考資料  
- https://qiita.com/ryuNagai/items/c5bda3f26da507ce37ea  
  実装したコードとその説明が書かれているサイト
- https://qiita.com/YuichiroMinato/items/16c634fc107ffa7bef82  
  日本語で書いてある参考サイト
- https://github.com/tensorflow/quantum/issues/180  
  動いた実績のあるバージョン確認
- https://www.tensorflow.org/quantum?hl=ja  
  tensorflow本家？の TensorFlow Quantum 説明ページ

In [1]:
!pip install tensorflow==2.1  # 2.1.0 でないと tensorflow_quantum が エラーで import できません

In [2]:
pip install tensorflow-quantum

In [3]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_quantum as tfq
print(tfq.__version__)

import cirq
print(cirq.__version__)
import sympy
import numpy as np

# 表示用
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

2.1.0
0.3.0
0.8.0


In [4]:
# 変更可能なパラメータ
p = 3 # QAOAのステップ数
K = 20 # 制約条件式のハイパーパラメータ
command_param = 0.1 # ニューラルネットの入力、多分何でも良い

 # Qubit数, 問題によって決まる
N = 6

# QAOA 回転角パラメータをsympy symbolとして用意
control_params = []
for i in range(p):
    gamma = 'gamma' + str(i)
    beta = 'beta' + str(i)
    control_params.append(sympy.symbols(gamma))
    control_params.append(sympy.symbols(beta))
  

In [5]:
# コストハミルトニアン時間発展
def CostUnitary(circuit, control_params, i, K):
    circuit.append(cirq.ZZ(qubits[0], qubits[1])**((K / 2 + 1) * control_params[i]))
    circuit.append(cirq.ZZ(qubits[0], qubits[2])**(K / 2 * control_params[i]))
    circuit.append(cirq.Z(qubits[0])**((-K / 2 - 6) * control_params[i]))
    circuit.append(cirq.ZZ(qubits[1], qubits[2])**((K + 1) / 2 * control_params[i]))
    circuit.append(cirq.Z(qubits[1])**((- K / 2 - 7) * control_params[i]))
    circuit.append(cirq.Z(qubits[2])**((- K / 2 - 5) * control_params[i]))
    circuit.append(cirq.ZZ(qubits[3], qubits[4])**((K / 2 + 1) * control_params[i]))
    circuit.append(cirq.ZZ(qubits[3], qubits[5])**(K / 2 * control_params[i]))
    circuit.append(cirq.Z(qubits[3])**((- K / 2 - 6) * control_params[i]))
    circuit.append(cirq.ZZ(qubits[4], qubits[5])**((K + 1) / 2 * control_params[i]))
    circuit.append(cirq.Z(qubits[4])**((- K / 2 - 7) * control_params[i]))
    circuit.append(cirq.Z(qubits[5])**((- K / 2 - 5) * control_params[i]))
    circuit.append(cirq.ZZ(qubits[0], qubits[3])**(2 * control_params[i]))
    circuit.append(cirq.ZZ(qubits[0], qubits[4])**(control_params[i]))
    circuit.append(cirq.ZZ(qubits[1], qubits[3])**(control_params[i]))
    circuit.append(cirq.ZZ(qubits[1], qubits[4])**(2 * control_params[i]))
    circuit.append(cirq.ZZ(qubits[1], qubits[5])**(1 / 2 * control_params[i]))
    circuit.append(cirq.ZZ(qubits[2], qubits[4])**(1 / 2 * control_params[i]))
    circuit.append(cirq.ZZ(qubits[2], qubits[5])**(2 * control_params[i]))
    return circuit

# ミキサハミルトニアン時間発展
def Mixer(circuit, i, N):
    for j in range(N):
        circuit.append(cirq.XPowGate(exponent = control_params[i]).on(qubits[j]))
    return circuit

# 初期状態(|+>)準備
def init_circuit(N):
    initCircuit = cirq.Circuit()
    qubits = cirq.GridQubit.rect(1, N)
    for j in range(N):
        initCircuit.append(cirq.H(qubits[j]))
    return initCircuit

In [6]:
qubits = [cirq.GridQubit(i, 0) for i in range(N)]
model_circuit = cirq.Circuit()
for i in range(p):
    model_circuit = CostUnitary(model_circuit, control_params, 2 * i, K)
    model_circuit = Mixer(model_circuit, 2 * i + 1, N)

In [7]:
def ZZoperator(i, j):
    return cirq.Z(qubits[i]) * cirq.Z(qubits[j])

operators = [[-1 * ((K / 2 + 1) * ZZoperator(0, 1) + (K / 2) * ZZoperator(0, 2) - \
            (K / 2 + 6) * cirq.Z(qubits[0]) + (K / 2 + 1 / 2) * ZZoperator(1, 2) - \
            (K / 2 + 7) * cirq.Z(qubits[1]) - (K / 2 + 5) * cirq.Z(qubits[2]) + \
            (K / 2 + 1) * ZZoperator(3, 4) + (K / 2) * ZZoperator(3, 5) - \
            (K / 2 + 6) * cirq.Z(qubits[3]) + (K / 2 + 1 / 2) * ZZoperator(4, 5) - \
            (K / 2 + 7) * cirq.Z(qubits[4]) - (K / 2 + 5) * cirq.Z(qubits[5]) + \
            2 * ZZoperator(0, 3) + ZZoperator(0, 4) + ZZoperator(1, 3) + 2 * ZZoperator(1, 4) +\
            1 / 2 * ZZoperator(1, 5) + 1 / 2 * ZZoperator(2, 4) + 2 * ZZoperator(2, 5) + 2 * K + 24)]]

In [8]:
# QAOAパラメータ最適化用のネットワークを定義
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(20 * p, activation='elu'),
    tf.keras.layers.Dense(len(control_params))
])

init_circuits = tfq.convert_to_tensor([init_circuit(N)])

commands_input = tf.keras.layers.Input(shape=(1),
                                       dtype=tf.dtypes.float32,
                                       name='commands_input')
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.dtypes.string,
                                name='circuits_input')
operators_input = tf.keras.Input(shape=(1,),
                                 dtype=tf.dtypes.string,
                                 name='operators_input')

dense_2 = controller(commands_input) # tf.keras.Sequential

full_circuit = tfq.layers.AddCircuit()(circuits_input, append=model_circuit)

expectation_output = tfq.layers.Expectation()(full_circuit,
                                              symbol_names=control_params,
                                              symbol_values=dense_2,
                                              operators=operators_input)

model = tf.keras.Model(
    inputs=[circuits_input, commands_input, operators_input],
    outputs=[expectation_output])

operator_data = tfq.convert_to_tensor(operators)
commands = np.array([[command_param] for i in range(len(operators))], dtype=np.float32)
expected_outputs = np.array([[0] for i in range(len(operators))], dtype=np.float32)

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()

model.compile(optimizer=optimizer, loss=loss)

In [10]:
history = model.fit(
    x=[init_circuits, commands, operator_data],
    y=expected_outputs,
    epochs=200,
    verbose=1)

Train on 1 samples
Epoch 1/200
1/1 [==============================] - 2s 2s/sample - loss: 1024.5895
Epoch 2/200
1/1 [==============================] - 0s 24ms/sample - loss: 11595.2568
Epoch 3/200
1/1 [==============================] - 0s 20ms/sample - loss: 3067.0828
Epoch 4/200
1/1 [==============================] - 0s 19ms/sample - loss: 2725.3796
Epoch 5/200
1/1 [==============================] - 0s 20ms/sample - loss: 19898.3887
Epoch 6/200
1/1 [==============================] - 0s 20ms/sample - loss: 1793.5094
Epoch 7/200
1/1 [==============================] - 0s 19ms/sample - loss: 1032.0602
Epoch 8/200
1/1 [==============================] - 0s 19ms/sample - loss: 4604.0000
Epoch 9/200
1/1 [==============================] - 0s 20ms/sample - loss: 5373.7012
Epoch 10/200
1/1 [==============================] - 0s 19ms/sample - loss: 7870.2451
Epoch 11/200
1/1 [==============================] - 0s 26ms/sample - loss: 2941.0405
Epoch 12/200
1/1 [==============================] - 0s 

In [11]:
print('Expectation value', model([init_circuits, commands, operator_data]))
after_params = controller.predict(np.array([command_param]))[0]
print('after params: ', after_params)

Expectation value tf.Tensor([[-86.49483]], shape=(1, 1), dtype=float32)
after params:  [ 1.7945818  -3.9364386   1.3545412   1.6278745  -3.6899114  -0.59774673]


In [12]:
param_dict = {}
for i in range(len(control_params)):
    param_dict.update([(control_params[i], after_params[i])])
resolver = cirq.ParamResolver(param_dict)

simulator = cirq.Simulator()
model_circuit.append(cirq.measure(*qubits, key = 'm'))
results = simulator.run(model_circuit, resolver, repetitions=100)

def calc_cost(state, operator):
    qubits = [cirq.GridQubit(i, 0) for i in range(len(state))]
    test_circuit = cirq.Circuit()
    for i in range(len(state)):
        if state[i] == '1':
            test_circuit.append(cirq.X(qubits[i]))
        else:
            test_circuit.append(cirq.I(qubits[i]))

    output_state_vector = cirq.Simulator().simulate(test_circuit).final_state
    qubit_map={qubits[0]: 0, qubits[1]: 1, qubits[2]: 2, qubits[3]: 3, qubits[4]: 4, qubits[5]: 5}
    return operator.expectation_from_wavefunction(output_state_vector, qubit_map).real

hist = results.histogram(key='m')
keys = list(hist.keys())
print('{:6}'.format('state'), '|', '{}'.format('count'), '|', '{}'.format('cost'))
for key in keys:
    binary = '{:0=6b}'.format(key)
    count = '{:5}'.format(hist[key])
    print(binary, '|',  count, '|', calc_cost(binary, operators[0][0]))

state  | count | cost
001011 |     1 | -44.0
001001 |    15 | -16.0
111001 |     2 | -112.0
110001 |     1 | -38.0
100100 |     8 | -16.0
011001 |     2 | -44.0
111111 |    21 | -232.0
010111 |     1 | -116.0
100110 |     3 | -48.0
010110 |     1 | -48.0
110010 |     2 | -48.0
111110 |     3 | -156.0
110111 |     2 | -156.0
011110 |     1 | -76.0
010101 |     1 | -38.0
001000 |     2 | -24.0
010000 |     3 | -24.0
110011 |     1 | -76.0
110110 |     3 | -88.0
000001 |     1 | -24.0
101101 |     4 | -72.0
001101 |     1 | -40.0
100000 |     2 | -24.0
001110 |     2 | -38.0
010010 |     5 | -16.0
001010 |     2 | -10.0
101011 |     3 | -72.0
101001 |     2 | -40.0
110100 |     2 | -48.0
000111 |     1 | -118.0
000010 |     1 | -24.0
000000 |     1 | -40.0
